<a href="https://colab.research.google.com/github/ev-greshkina/DataAnalyst-educational-course/blob/main/%D0%A0%D0%B5%D1%88%D0%B5%D0%BD%D0%B8%D0%B5_%D0%94%D0%97_%D0%BD%D0%B0_%D1%82%D0%B5%D0%BC%D1%83_%22onlineShopping_c_ipynb%22_v_1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Анализ логов продуктового магазина

Представьте, что вы работаете аналитиком онлайн-магазина и ваша цель - повысить GMV (суммарную выручку от продажи товаров). Для этого вы хотите провести ряд исследований и придумать какие-то меры для увеличения выручки.

* Определите, какого числа было сделано больше всего заказов?
* Найдите топ-3 ParentCategory по GMV (суммарная выручка, которую принесла продажа товаров этой категории)
* Напишите функцию, которая принимает на вход OrderID и рекомендует ItemID продукта для корзины этого заказа. Проверьте работу этой функции.

In [ ]:
import pandas as pd
import datetime

In [ ]:
# Для скачивания файлов из репозитория в Colab
!wget https://raw.githubusercontent.com/da-python/new-modul-python/main/Lesson4/hometask/data/categories.csv -P data
!wget https://raw.githubusercontent.com/da-python/new-modul-python/main/Lesson4/hometask/data/orders.json -P data

--2024-09-16 19:05:53--  https://raw.githubusercontent.com/da-python/new-modul-python/main/Lesson4/hometask/data/categories.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2246830 (2.1M) [text/plain]
Saving to: ‘data/categories.csv’

categories.csv      100%[===================>]   2.14M  --.-KB/s    in 0.05s   

2024-09-16 19:05:53 (41.7 MB/s) - ‘data/categories.csv’ saved [2246830/2246830]

--2024-09-16 19:05:53--  https://raw.githubusercontent.com/da-python/new-modul-python/main/Lesson4/hometask/data/orders.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent,

In [ ]:
categories = pd.read_csv('data/categories.csv')
orders = pd.read_json('data/orders.json')

In [ ]:
categories.head()

,ItemID,Category,ParentCategory
0,1000002,Сладости для диабетиков,Сладости
1,1000046,Вода,Напитки
2,1000048,Сладости для диабетиков,Сладости
3,1000049,Вода,Напитки
4,1000061,Фрукты,Овощи и фрукты


In [ ]:
orders.head()

,OrderID,UserID,OrderTime,items,Date
0,90659865,1.609460e+18,2021-01-01 03:51:42,"[{'ItemID': '413736', 'price': 36.72, 'quantit...",2021-01-01
1,90622054,2.499293e+18,2021-01-01 04:13:13,"[{'ItemID': '961454', 'price': 179.9, 'quantit...",2021-01-01
2,90659952,3.945866e+17,2021-01-01 05:48:53,"[{'ItemID': '306553', 'price': 101.9, 'quantit...",2021-01-01
3,89765281,5.412765e+18,2021-01-01 08:33:41,"[{'ItemID': '979703', 'price': 99.9, 'quantity...",2021-01-01
4,90582308,6.380061e+18,2021-01-01 09:55:35,"[{'ItemID': '303765', 'price': 44.9, 'quantity...",2021-01-01


In [ ]:
# Решение задания 1 (Определите, какого числа было сделано больше всего заказов?)
# Сгруппировать даты покупок и определить сколько раз эти даты встречаются.
# Вывод результата в виде Series, в котором индексом является дата,
# а значением является количество раз, сколько эта дата встречается.
max_orders_date = orders['Date'].value_counts()

# Создать колонку Date и скопировать в нее значения индексов
max_orders_date = max_orders_date.reset_index()

# Преобразовать столбец с данными типа Datestamp в тип Datatime
max_orders_date['Date'] = pd.to_datetime(max_orders_date['Date'])

# Вывести результат (первое значение столбца Date и столбца count)
print("Наибольшее количество покупок (", max_orders_date.loc[0, 'count'], \
      ') сделано ', max_orders_date.loc[0, 'Date'].date())

Наибольшее количество покупок ( 342 ) сделано  2021-02-05


In [35]:
# Решение задания 2 (Найдите топ-3 ParentCategory по GMV (суммарная выручка, которую принесла продажа товаров этой категории))

# Получить все значения столбца items и собрать их в список словарей

# Создать необходимые переменные:
# переменная счетчик для управления циклом перебора всех строк таблицы
row_index = 0

# Пустой список для записи значений ячеек столбца items
all_items = []

# Перебор в цикле всех строк таблица, выбор на каждой итерации ячеки столбца
# items и добавление из в список all_items
#for ind in df.index:
    #print(df['Name'][ind], df['Stream'][ind])
for row in orders.index:
  all_items.extend(orders.loc[row, 'items'])

# Преобразовать список словарей в Dataframe объект, где столбцами будут ключи
# словарей, а значениями будут значения ячеек
df_all_items = pd.DataFrame(all_items)

# Перед объединением в столбец ItemID в таблице df_all_items привести в тип данных int64
# для дальнейшего объединения
df_all_items.ItemID = df_all_items.ItemID.astype('int64')

# Добавить столбец cost содержащий сумму затрат на приобретение товара (значение
# столбца price умноженное на quantity)
df_all_items_with_cost = df_all_items.assign(cost = df_all_items['price'] * df_all_items['quantity'])

# Объединить таблицы df_all_items и categories по значениям столбцов ItemID
all_items_with_categories = df_all_items_with_cost.merge(categories, left_on='ItemID', right_on ='ItemID', suffixes=('_left', '_right'))

# Создать новый DataFrame, содержащий столбцы cost и ParentCategory
cost_of_parent_categorie = all_items_with_categories[['cost', 'ParentCategory']]

# Сгруппировать по столбцу ParentCategory
grouped_items = cost_of_parent_categorie.groupby('ParentCategory')

# Суммировать группы
aggregated_items = grouped_items.agg({'cost': 'sum'})

# Пребразовать index в столбец
result = aggregated_items.reset_index()

# Сортировать DataFrame в порядке убывания
result = result.sort_values('cost', ascending = False)

# Переиндексация строк для получения последовательности индексов по возростанию
result = result.reset_index()

print(f"Топ-3 ParentCategory по GMV (суммарная выручка, которую принечли продажа \
товаров этой категории):\n- {result.loc[0, 'ParentCategory']} ({result.loc[0, 'cost']}),\n\
- {result.loc[1, 'ParentCategory']} ({result.loc[1, 'cost']}),\n- {result.loc[2, 'ParentCategory']} ({result.loc[2, 'cost']})")

Топ-3 ParentCategory по GMV (суммарная выручка, которую принечли продажа товаров этой категории):
- Мясо, птица, рыба (32268493.46),
- Молочная продукция (27702995.43),
- Овощи и фрукты (27607486.900000002)


In [ ]:
# Задание 3. Напишите функцию, которая принимает на вход OrderID и рекомендует ItemID
# продукта для корзины этого заказа. Проверьте работу этой функции. (93955735)

# Запрос у пользователя выбрать OrderID с проверкой правильности введенного значения
while True:
  try:
    user_order = int(input('Введите номер заказа: '))
    break
  except ValueError:
    print("Введенное Вами значение недопустимо, введите корректное значение.")


# Функция для выямсления рекомендаций
def recomented_items(user_order):
  # Создать переменную для хранения значений из столбца Items
  user_items = []

  # Создать переменную для управления циклом
  row_index = 0

  # Получить строку таблицы, в котрая содержит искомый OrderID
  target_row = orders.loc[orders['OrderID'] == user_order]

  # Получить индекс покупателя, выделив значение из ячейки
  target_user = target_row['UserID']

  # Получить все строки, в которых есть полученный UserID
  # Собрать значения всех ячеек в список словарей
  all_users_orders = orders.loc[orders['UserID'] == int(target_user.iloc[0])]

  # Сбросить параметры индекциии объекта
  all_users_orders = all_users_orders.reset_index()

  # Получить все значения колонки items в виде списка словарей
  for row in all_users_orders.iterrows():
    user_items.extend(all_users_orders.loc[row_index, 'items'])
    row_index += 1

  # Преборазовать полученный список в DataFrame, где ключи это названия колонок.
  user_items = pd.DataFrame(user_items)

  # Сгруппировать и строки по столбцу ItemID с подсчетом количества раз, сколько
  # встречается это значение (применить функцию value_counts)
  groupped_user_items = user_items['ItemID'].value_counts()

  # Провести индексацию полученного результата
  groupped_user_items = groupped_user_items.reset_index()

  # Изменить тип данных объект groupped_user_items для объединения таблиц
  groupped_user_items['ItemID'] = groupped_user_items['ItemID'].astype(int)

  # Объединить полученную таблицу с таблицей Categories
  union_tables = groupped_user_items.merge(categories, left_on = 'ItemID',
                                           right_on = 'ItemID',
                                           suffixes=('_left', '_right'))

  # Альтернативный вариант получения рекомендация, если товаров меньше 5
  recomendations = union_tables.head(5)

  # Получить список рекомендованных идентификаторов товаров
  recomendations_list = recomendations['ItemID'].tolist()

  # Преобразовать спико рекомендаций в строку с разделителем "," для
  # вставки в итоговое сообение.
  recomendations_list_str = ', '.join([str(elem) for elem in recomendations_list])

  # Вывести рекомендацию наиболее часто покупаемых товаров.
  print(f"На основе Ваших покупок рекомендуем товары с идентификаторами: \
  {recomendations_list_str}.")

# Вызов функции
recomented_items(user_order)

Введите номер заказа: 93955735
На основе Ваших покупок рекомендуем товары с идентификаторами   307209, 307927, 430560, 309474, 997675.
